In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
from datetime import datetime

### Сбор данных с сайта auto.ru

In [ ]:
# Собираем список url авто
def get_car_url_list(page_url_n, numbers, tag_1, tag_2, class_):
    
    car_url_list = []
    
    for x in brand_low:
        try:
            url = page_url_n+str(x)+'/used/'
            page = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')

            a_blocks = page.find_all(tag_1, class_=class_)
            for block in a_blocks:
                link = block.get(tag_2)
                car_url_list.append(link)

            for n in numbers:
                try:
                    url = page_url_n+str(x)+'/used/?page='+str(n)
                    page_n = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')

                    a_blocks = page_n.find_all(tag_1, class_=class_)
                    for block in a_blocks:
                        link = block.get(tag_2)
                        car_url_list.append(link)
                except: continue
        
        except: continue
    
    return car_url_list

In [ ]:
brand = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI', 'MERCEDES',
         'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
brand_low = []
for x in brand:
    brand_low.append(x.lower())

In [ ]:
# выдает максимум по 99 страниц
numbers = range(2,100)
page_url_n = 'https://auto.ru/cars/'
tag_1 = 'a'
tag_2 = 'href'
class_ = 'Link ListingItemTitle-module__link'

In [ ]:
all_cars_url = get_car_url_list(page_url_n, numbers, tag_1, tag_2, class_)

In [ ]:
print(len(all_cars_url))
print(all_cars_url[1])
print(type(all_cars_url))

In [ ]:
# Создаем списки, из которых потом будем делать столбцы дата фрейма
# sell_id_list = []
car_title_list = []
price_list = []
priceCurrency_list = []
city_list = []
CardInfo_param_dict_list = []
# CardDescription_len_list = []
CardDescription_list = []
Complectation_list = []
# Complectation_list_len = []

In [ ]:
# Читаем страницы авто и заполняем списки
def get_car_param(car_urls):
    
    for car_url in car_urls:
        page_car_x = requests.get(car_url, headers={'User-Agent': 'Mozilla/5.0'})
        page_car_x.encoding = 'utf-8'
        page_car = BeautifulSoup(page_car_x.text, 'html.parser')
        
        # id авто
        # try:
        #     sell_id_list.append(int(car_url.split('/')[-2][:10]))
        # except:
        #     sell_id_list.append(np.nan)

        # Имя авто
        try:
            title_block = page_car.find('div', class_='CardSidebarActions__title')
            car_title_list.append(title_block.text)
        except:
            car_title_list.append(np.nan)

        # Цена авто
        try:
            price_block = page_car.find('span', class_='OfferPriceCaption__price')
        #     price_list.append(float(price_block.text.replace(u'\xa0', '')[:-1]))
        #     priceCurrency_list.append(price_block.text.replace(u'\xa0', '')[-1])
            price_list.append(price_block.text[:-1])
            priceCurrency_list.append(price_block.text[-1])
        except:
            price_list.append(np.nan)
            priceCurrency_list.append(np.nan)

        # Город продавца
        try:
            city_block = page_car.find('span', class_='MetroListPlace__regionName MetroListPlace_space')
            city_list.append(city_block.text)
        except:
            city_list.append(np.nan)

        # Длина описания
        try:
            descript_block = page_car.find('div', class_='CardDescription__textInner')
        #     CardDescription_len_list.append(len(descript_block.text))
            CardDescription_list.append(descript_block.text)
        except:
        #     CardDescription_len_list.append(np.nan)
            CardDescription_list.append(np.nan)

        # equipment_dict
        try:
            complect_block = page_car.find_all('li', class_='ComplectationGroups__itemContentEl')
            complect_list = []
            for item in complect_block:
        #         complect_list.append(item.text.lower())
                complect_list.append(item.text)
            complect_dict = {key: value for key, value in zip(complect_list, ['true' for x in complect_list])}
            Complectation_list.append(complect_dict)
        #     Complectation_list_len.append(len(complect_list))
        except:
            Complectation_list.append(np.nan)
        #     Complectation_list_len.append(np.nan)

        # набор параметров может быть разный, поэтому достаем связку
        try:
            all_span = page_car.find_all('span', class_='CardInfoRow__cell')
            all_param = []
            for item in all_span:
        #         all_param.append(item.text.replace(u'\xa0', ' ').lower())
                all_param.append(item.text)
    
            CardInfo_param_key = []
            CardInfo_param_value = []
            for param in all_param[0::2]:
                CardInfo_param_key.append(param)
            for param in all_param[1::2]:
                CardInfo_param_value.append(param)
    
            CardInfo_param_dict = {key: value for key, value in zip(CardInfo_param_key,CardInfo_param_value)}
            CardInfo_param_dict_list.append(CardInfo_param_dict)
        except:
            CardInfo_param_dict_list.append(np.nan)
        
    return np.array(car_urls), np.array(car_title_list), np.array(price_list), np.array(priceCurrency_list), np.array(city_list), np.array(CardInfo_param_dict_list), np.array(CardDescription_list), np.array(Complectation_list)

In [ ]:
car_urls = all_cars_url

In [ ]:
a = get_car_param(car_urls)

In [ ]:
np.shape(a)

In [ ]:
a_list = np.array([el for el in a])
print(type(a_list))
print(len(a_list[7]))
print(np.shape(a_list.T))

In [ ]:
auto_ru_data = pd.DataFrame(a_list.T,
                            columns = ['car_urls', 'car_title', 'price', 'price_currency', 'city'
                                       , 'car_param_dict', 'car_description'
                                       , 'car_complectation'])

In [ ]:
auto_ru_data['parsing_date'] = pd.to_datetime(datetime.today().date())

In [ ]:
auto_ru_data.head()

In [ ]:
for column in auto_ru_data.columns:
    print(column+': '+str(auto_ru_data[column][2]))

In [ ]:
auto_ru_data.info()

In [ ]:
len(auto_ru_data['car_urls'].unique())

In [ ]:
auto_ru_data.to_csv('auto_ru_data_2part.csv', index=False)